In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams['image.cmap'] = 'gray'
import cv2
import cv2 as cv
import numpy as np
import dlib
import simpleaudio as sa
from scipy.spatial import distance as dist

EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 60


def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear


cap = cv2.VideoCapture(0)
import cv2 as cv
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
counter=0
COUNTER=0
countdown=0
countd=0
filename = 'StarWars3.wav'
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    if countd!=0:
        cv2.putText(frame, "Look on the road", (200, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        countd-=1
    if len(faces)==0:
        counter+=1
        if counter==10:
            
            wave_obj = sa.WaveObject.from_wave_file(filename)
            play_obj = wave_obj.play()
            play_obj.wait_done()  
            countd=60

            
    else:
        counter=0
        for face in faces:
            x1 = face.left()
            y1 = face.top()
            x2 = face.right()
            y2 = face.bottom()
            #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

            landmarks = predictor(gray, face)
            l=[]
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
#                 cv2.circle(frame, (x, y), 1, (255, 0, 0), -1)
                l.append((x,y))
            
            leye=l[36:42]
            reye=l[42:48]
            leye=np.array(leye)
            reye=np.array(reye)
            leftEyeHull = cv2.convexHull(leye)
            rightEyeHull = cv2.convexHull(reye)
            mask = np.zeros((frame.shape[0], frame.shape[1]))
            mask=cv2.fillConvexPoly(mask, leftEyeHull, 1)
            mask=cv2.fillConvexPoly(mask, rightEyeHull, 1)
            
            o = mask.astype(np.bool)
            
            out = np.zeros_like(frame)
            out[o] = frame[o]
            gray1 = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
            gray1 = cv2.medianBlur(gray1,5)
            
            
            leftEAR = eye_aspect_ratio(leye)
            rightEAR = eye_aspect_ratio(reye)
            ear = (leftEAR + rightEAR) / 2.0
            
            if ear<EYE_AR_THRESH:
                COUNTER += 1
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    COUNTER=0
                    wave_obj = sa.WaveObject.from_wave_file(filename)
                    play_obj = wave_obj.play()
                    play_obj.wait_done() 
                    countdown=60
        if countdown!=0:
            cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            countdown-=1
        
        
    cv2.imshow("Frame", frame)
    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

    
cap.release()
cv2.destroyAllWindows()